# 中文分词引擎 java 实现


# 正向最大匹配法
分词目标：

在词典中进行扫描，尽可能地选择与词典中最长单词匹配的词作为目标分词，然后进行下一次匹配。


算法流程：

假设词典中最长的单词为 5 个（MAX_LENGTH），那么最大匹配的起始子串字数也为 5 个

（1）扫描字典，测试读入的子串是否在字典中

（2）如果存在，则从输入中删除掉该子串，重新按照规则取子串，重复（1）

（3）如果不存在于字典中，则从右向左减少子串长度，重复（1）


In [ ]:
public List<String> leftMax(String str) {

        List<String> results = new ArrayList<String>();
        String input = str;

        while( input.length() > 0 ) {

            String subSeq;
            // 每次取小于或者等于最大字典长度的子串进行匹配
            if( input.length() < MAX_LENGTH) 
                subSeq = input;
            else
                subSeq = input.substring(0, MAX_LENGTH);

            while( subSeq.length() > 0 ) {
                // 如果字典中含有该子串或者子串颗粒度为1，子串匹配成功
                if( dictionary.contains(subSeq) || subSeq.length() == 1) {
                    results.add(subSeq);
                    // 输入中从前向后去掉已经匹配的子串
                    input = input.substring(subSeq.length());
                    break;      // 退出循环，进行下一次匹配
                } else {
                    // 去掉匹配字段最后面的一个字
                    subSeq = subSeq.substring(0, subSeq.length() - 1);
                }   
            }

        }
        return results;
    }

# 逆向匹配法

In [ ]:
public List<String> rightMax(String str) {
        // 采用堆栈处理结果，后进先出
        Stack<String> store=new Stack<String>();
        List<String> results = new ArrayList<String>();
        String input = str;

        while( input.length() > 0 ) {

            String subSeq;
            // 每次取小于或者等于最大字典长度的子串进行匹配
            if( input.length() < MAX_LENGTH)
                subSeq = input;
            else 
                subSeq = input.substring(input.length() - MAX_LENGTH);

            while( subSeq.length() > 0 ) {
                // 如果字典中含有该子串或者子串颗粒度为1，子串匹配成功
                if( dictionary.contains(subSeq) || subSeq.length() == 1) {
                    store.add(subSeq);
                    // 输入中从后向前去掉已经匹配的子串
                    input = input.substring(0, input.length() - subSeq.length());
                    break;
                } else {
                    // 去掉匹配字段最前面的一个字
                    subSeq = subSeq.substring(1);
                }
            }
        }
        // 输出结果
        int size = store.size();
        for( int i = 0; i < size; i ++) {
            results.add(store.pop());
        }

        return results;
    }

# 双向最大匹配法
分词目标：

将正向最大匹配算法和逆向最大匹配算法进行比较，从而确定正确的分词方法。


算法流程：

比较正向最大匹配和逆向最大匹配结果
如果分词数量结果不同，那么取分词数量较少的那个
如果分词数量结果相同
分词结果相同，可以返回任何一个
分词结果不同，返回单字数比较少的那个


分词实例：

就上例来看，

正向匹配最终切分结果为：北京大学 / 生前 / 来 / 应聘，分词数量为 4，单字数为 1

逆向匹配最终切分结果为：”北京/ 大学生/ 前来 / 应聘，分词数量为 4，单字数为 0

逆向匹配单字数少，因此返回逆向匹配的结果。

双向最大匹配法实现如下：


In [ ]:
public List<String> segment() {
        List<String> fmm = this.leftMax();
        List<String> bmm = this.rightMax();

        // 如果分词的结果不同，返回长度较小的
        if( fmm.size() != bmm.size()) {
            if ( fmm.size() > bmm.size())
                return bmm;
            else 
                return bmm;
        }
        // 如果分词的词数相同
        else {
            int fmmSingle = 0, bmmSingle = 0;
            boolean isEqual = true;
            for( int i = 0; i < bmm.size(); i ++) {
                if( !fmm.get(i).equals(bmm.get(i))) {
                    isEqual = false;
                }
                if( fmm.get(i).length() == 1)
                    fmmSingle ++;
                if( bmm.get(i).length() == 1)
                    bmmSingle ++;
            }
            // 如果正向、逆向匹配结果完全相等，返回任意结果
            if ( isEqual ) {
                return fmm;
            // 否则，返回单字数少的匹配方式
            } else if ( fmmSingle > bmmSingle)      
                return bmm;
            else 
                return fmm;     
        }

    }

# viterbi 算法
链接：https://www.zhihu.com/question/20136144

In [ ]:
import numpy as np
def viterbi(trainsition_probability,emission_probability,pi,obs_seq):
    #转换为矩阵进行运算
    trainsition_probability=np.array(trainsition_probability)
    emission_probability=np.array(emission_probability)
    pi=np.array(pi)
    obs_seq = [0, 2, 3]
    # 最后返回一个Row*Col的矩阵结果
    Row = np.array(trainsition_probability).shape[0]
    Col = len(obs_seq)
    #定义要返回的矩阵
    F=np.zeros((Row,Col))
    #初始状态
    F[:,0]=pi*np.transpose(emission_probability[:,obs_seq[0]])
    for t in range(1,Col):
        list_max=[]
        for n in range(Row):
            list_x=list(np.array(F[:,t-1])*np.transpose(trainsition_probability[:,n]))
            #获取最大概率
            list_p=[]
            for i in list_x:
                list_p.append(i*10000)
            list_max.append(max(list_p)/10000)
        F[:,t]=np.array(list_max)*np.transpose(emission_probability[:,obs_seq[t]])
    return F

if __name__=='__main__':
    #隐藏状态
    invisible=['Sunny','Cloud','Rainy']
    #初始状态
    pi=[0.63,0.17,0.20]
    #转移矩阵
    trainsion_probility=[[0.5,0.375,0.125],[0.25,0.125,0.625],[0.25,0.375,0.375]]
    #发射矩阵
    emission_probility=[[0.6,0.2,0.15,0.05],[0.25,0.25,0.25,0.25],[0.05,0.10,0.35,0.5]]
    #最后显示状态
    obs_seq=[0,2,3]
    #最后返回一个Row*Col的矩阵结果
    F=viterbi(trainsion_probility,emission_probility,pi,obs_seq)
    print(F)